# Lab 5: Choice Algorithm
11/15/2024 - Adam Haile, Aiden Miller, Leigh Goetsch, Class: CSC3310

In [ ]:
import math
import random
from typing import Any, List

In [ ]:
class Node():
    def __init__(self, value: Any, parent: 'Node' = None, left: 'Node' = None, right: 'Node' = None):
        self.value = value
        self.parent = parent
        self.left = left
        self.right = right

In [ ]:
import sys
import math
from typing import Any, List

sys.setrecursionlimit(1500)

class Node():
    def __init__(self, value: Any, parent: 'Node' = None, left: 'Node' = None, right: 'Node' = None):
        self.value = value
        self.parent = parent
        self.left = left
        self.right = right

class SGTree:
    def __init__(self, root: Any = None):
        self.root = Node(root) if root is not None else None
        self.alpha = 2 / 3
        self._num_nodes= 0

    def __len__(self) -> int:
        return self.size()

    def _bst_insert_with_depth(self, u: Node) -> int:
        w = self.root
        if w is None:
            self.root = u
            self._num_nodes+= 1
            return 0
        
        done = False
        d = 0
        while not done:
            if u.value < w.value:
                if w.left is None:
                    w.left = u
                    u.parent = w
                    done = True
                else:
                    w = w.left
            elif u.value > w.value:
                if w.right is None:
                    w.right = u
                    u.parent = w
                    done = True
                else:
                    w = w.right
            else:
                # Duplicate value, insertion fails
                return -1
            d += 1

        self._num_nodes+= 1
        return d
    
    def _inorder_collect(self, node: Node, nodes: List[Node]) -> None:
        if node is None:
            return
        self._inorder_collect(node.left, nodes)
        nodes.append(node)
        self._inorder_collect(node.right, nodes)

    def _build_balanced_tree(self, nodes: List[Node], start: int, end: int) -> Node:
        if start > end:
            return None
        mid = (start + end) // 2
        root = nodes[mid]
        root.left = self._build_balanced_tree(nodes, start, mid - 1)
        root.right = self._build_balanced_tree(nodes, mid + 1, end)
        
        if root.left:
            root.left.parent = root
        if root.right:
            root.right.parent = root
        
        return root

    def _rebuild_subtree(self, scapegoat_node: Node) -> Node:
        nodes = []
        self._inorder_collect(scapegoat_node, nodes)
        return self._build_balanced_tree(nodes, 0, len(nodes) - 1)

    def insert(self, value: Any) -> bool:
        node = Node(value)
        depth = self._bst_insert_with_depth(node)

        # If insertion was unsuccessful (duplicate value), return False
        if depth < 0:
            return False

        # Check if tree needs rebalancing
        if depth > math.ceil(math.log(self._num_nodes, 1 / self.alpha)):
            p = node.parent

            # Find the scapegoat node
            while p.parent and 3 * self._size(p) <= 2 * self._size(p.parent):
                p = p.parent

            # Rebuild the subtree rooted at the scapegoat node
            rebuilt_subtree = self._rebuild_subtree(p)

            # Update parent's child pointers
            if p.parent is None:
                self.root = rebuilt_subtree
            else:
                if p.parent.left == p:
                    p.parent.left = rebuilt_subtree
                else:
                    p.parent.right = rebuilt_subtree

            # Update the parent pointer of the new subtree root
            rebuilt_subtree.parent = p.parent

        return True
    
    def _size(self, root: Node) -> int:
        if root is None:
            return 0
        stack = [root]
        size = 0
        while stack:
            node = stack.pop()
            if node:
                size += 1
                stack.append(node.left)
                stack.append(node.right)
        return size

    # def _size(self, root: Node) -> int:
    #     if root is None:
    #         return 0
    #     return 1 + self._size(root.left) + self._size(root.right)

    def size(self) -> int:
        return self._num_nodes
    
    def _contains(self, root: Node, target: Any) -> bool:
        if root is None:
            return False
        if root.value == target:
            return True
        if target < root.value:
            return self._contains(root.left, target)
        return self._contains(root.right, target)

    def contains(self, value: Any) -> bool:
        return self._contains(self.root, value)
    
    def _find_min(self, node: Node) -> Node:
        while node.left:
            node = node.left
        return node
    
    def _delete(self, node: Node, value: Any) -> Node:
        if not node:
            return None
        if value < node.value:
            node.left = self._delete(node.left, value)
        elif value > node.value:
            node.right = self._delete(node.right, value)
        else:
            if not node.left:
                return node.right
            if not node.right:
                return node.left
            successor = self._find_min(node.right)
            node.value = successor.value
            node.right = self._delete(node.right, successor.value)
        return node
    
    def delete(self, value: Any) -> None:
        m = self._num_nodes
        self.root = self._delete(self.root, value)
        self._num_nodes-= 1

        if self._num_nodes< self.alpha * m:
            self.root = self._rebuild_subtree(self.root)
    
    def to_list(self) -> List[Any]:
        nodes: List[Node] = []
        self._inorder_collect(self.root, nodes)
        return [node.value for node in nodes]


In [ ]:
values = random.sample(range(1, 101), 10)
values

In [ ]:
tree = SGTree()
for value in values:
    tree.insert(value)
    print(tree.to_list())

for value in random.sample(values, 10):
    tree.delete(value)
    print(tree.to_list())

## Tests

## Benchmarks
Benchmark the insert, delete, and contains operations of your implementation on data sets of
different sizes. Create tables and plots that include both run times and the number of times the
rebuild operation was performed.

In [ ]:
import random
import time

# benchmark insert operation
def benchmark_insert(scapegoat: SGTree, n: int) -> float:
    # copy the tree

    
    start = time.time()
    scapegoat.insert(n)
    return time.time() - start

# benchmark delete operation
def benchmark_delete(scapegoat: SGTree, n: int) -> float:
    # copy the tree


    start = time.time()
    scapegoat.delete(n)
    return time.time() - start


# benchmark contains operation
def benchmark_contains(scapegoat: SGTree, n: int) -> float:
    # copy the tree


    start = time.time()
    scapegoat.contains(n)
    return time.time() - start

In [ ]:
# set up benchmarking lists
tree_inputs = [(pow(2, n) - pow(2, n - 2)) for n in range(4, 20)]
benchmark_lists = {
    "sorted": [list(range(n)) for n in tree_inputs],
    "reverse-sorted": [list(range(n, 0, -1)) for n in tree_inputs],
    "random": [random.sample(range(n), n) for n in tree_inputs]
}

results = {
    "insert": {k: [] for k in benchmark_lists},
    "delete": {k: [] for k in benchmark_lists},
    "contains": {k: [] for k in benchmark_lists}
}

In [ ]:
for key in benchmark_lists:
    for input_list in benchmark_lists[key]:
        scapegoat = SGTree()
        for n in input_list:
            scapegoat.insert(n)
            # print("added", n)
        results["insert"][key].append(benchmark_insert(scapegoat, n))
        results["delete"][key].append(benchmark_delete(scapegoat, n))
        results["contains"][key].append(benchmark_contains(scapegoat, n))

In [ ]:
scapegoat.to_list()